In [318]:
import numpy as np
import pandas as pd

from scipy.stats import wilcoxon

from datasets import binclas_datasets, regr_datasets
from config import dataset_map

In [319]:
splits = {
    'dtc': pd.read_csv('splits_dtc.csv'),
    'dtr': pd.read_csv('splits_dtr.csv'),
    'rfc': pd.read_csv('splits_rfc.csv'),
    'rfr': pd.read_csv('splits_rfr.csv')
}
evaluations = {
    'dtc': pd.read_csv('evaluation_dtc.csv'),
    'dtr': pd.read_csv('evaluation_dtr.csv'),
    'rfc': pd.read_csv('evaluation_rfc.csv'),
    'rfr': pd.read_csv('evaluation_rfr.csv')
}

scores = {
    'dtc': 'auc',
    'rfc': 'auc',
    'dtr': 'r2',
    'rfr': 'r2'
}

binclas_order = binclas_datasets[['name']]\
    .apply(lambda row: pd.Series({'name': dataset_map.get(row['name'], row['name'])}), axis=1)
regr_order = regr_datasets[['name']]\
    .apply(lambda row: pd.Series({'name': dataset_map.get(row['name'], row['name'])}), axis=1)

In [320]:
splits = {
    key: value.rename(columns={'Unnamed: 0': 'name'})
    for key, value in splits.items()
}

In [321]:
splits['dtc']['name'] = splits['dtc']['name'].apply(lambda name: dataset_map.get(name, name))
splits['dtr']['name'] = splits['dtr']['name'].apply(lambda name: dataset_map.get(name, name))
splits['rfc']['name'] = splits['rfc']['name'].apply(lambda name: dataset_map.get(name, name))
splits['rfr']['name'] = splits['rfr']['name'].apply(lambda name: dataset_map.get(name, name))

In [322]:
splits['dtc']

,name,n_lattice_splits,n_splits,n_lattice_splits_kfold,n_splits_kfold
0,appendicitis,0,3,1,283
1,haberman,0,7,14,687
2,new_thyroid1,0,2,3,202
3,glass0,0,5,18,557
4,shuttle-6_vs_2-3,0,1,0,100
5,bupa,2,17,158,1652
6,cleveland-0_vs_4,0,3,4,279
7,ecoli1,0,3,17,300
8,poker-9_vs_7,0,2,0,223
9,monk-2,0,5,0,474


In [323]:
for key in splits:
    splits[key]['rate'] = splits[key]['n_lattice_splits'] / splits[key]['n_splits']
    splits[key]['rate_kfold'] = splits[key]['n_lattice_splits_kfold'] / splits[key]['n_splits_kfold']

In [324]:
evaluations = {
    key: value[value['mode'].isin(['<', '<='])]
    for key, value in evaluations.items()
}

In [325]:
evaluations = {
    key: value.groupby(['name', 'mode'])\
            .apply(lambda pdf: pdf.sort_values('fold')[scores[key]].values.tolist())\
            .reset_index(drop=False)\
            .rename(columns={0: scores[key]})
    for key, value in evaluations.items()
}

In [326]:
def evaluate(pdf):
    score = 'auc' if 'auc' in pdf.columns else 'r2'
    leq = pdf[pdf['mode'] == '<='].iloc[0]
    l = pdf[pdf['mode'] == '<'].iloc[0]

    p_less = wilcoxon(leq[score], l[score], zero_method='zsplit', alternative='less').pvalue
    p_gr = wilcoxon(leq[score], l[score], zero_method='zsplit', alternative='greater').pvalue

    return pd.Series({
        f'{score}_diff': np.mean(leq[score]) - np.mean(l[score]),
        'p': min(p_less, p_gr)
    })

In [327]:
figures = {
    key: value.groupby('name').apply(evaluate)
    for key, value in evaluations.items()
}

In [328]:
joined = {
    key: pd.merge(value.reset_index(drop=False), splits[key][['name', 'rate', 'rate_kfold']], on=['name'])\
            .set_index('name')
    for key, value in figures.items()
}

In [329]:
print(joined['dtc']), binclas_order

                        auc_diff             p      rate  rate_kfold
name                                                                
abalone9_18             0.000000  5.000000e-01  0.000000    0.000000
appendicitis            0.000000  5.000000e-01  0.000000    0.003534
bupa                    0.000105  7.051001e-03  0.117647    0.095642
cleveland-0_vs_4       -0.003139  7.374231e-02  0.000000    0.014337
ecoli1                  0.000000  5.000000e-01  0.000000    0.056667
glass0                  0.000000  5.000000e-01  0.000000    0.032316
haberman               -0.000335  4.971797e-04  0.000000    0.020378
hepatitis              -0.000244  4.734861e-01  0.000000    0.044077
lymphography           -0.003967  1.546562e-01  0.000000    0.000000
mammographic            0.000026  3.769766e-03  0.000000    0.001086
monk-2                  0.000000  5.000000e-01  0.000000    0.000000
new_thyroid1            0.000004  4.119086e-01  0.000000    0.014851
page-blocks-1-3_vs_4    0.000149  

(None,
                       name
 0             appendicitis
 1                 haberman
 2             new_thyroid1
 3                   glass0
 4         shuttle-6_vs_2-3
 5                     bupa
 6         cleveland-0_vs_4
 7                   ecoli1
 8             poker-9_vs_7
 9                   monk-2
 10               hepatitis
 11    yeast-0-3-5-9_vs_7-8
 12            mammographic
 13                 saheart
 14    page-blocks-1-3_vs_4
 15            lymphography
 16                    pima
 17               wisconsin
 18             abalone9_18
 19  winequality-red-3_vs_5)

In [330]:
joined['dtc'] = pd.merge(binclas_order, joined['dtc'], on=['name']).set_index('name')
joined['rfc'] = pd.merge(binclas_order, joined['rfc'], on=['name']).set_index('name')
joined['dtr'] = pd.merge(regr_order, joined['dtr'], on=['name']).set_index('name')
joined['rfr'] = pd.merge(regr_order, joined['rfr'], on=['name']).set_index('name')

In [331]:
joined['dtc'].columns = pd.MultiIndex.from_tuples(('Decision Tree', col) for col in joined['dtc'].columns)
joined['rfc'].columns = pd.MultiIndex.from_tuples(('Random Forest', col) for col in joined['rfc'].columns)
joined['dtr'].columns = pd.MultiIndex.from_tuples(('Decision Tree', col) for col in joined['dtr'].columns)
joined['rfr'].columns = pd.MultiIndex.from_tuples(('Random Forest', col) for col in joined['rfr'].columns)

In [332]:
binclas = pd.merge(joined['dtc'], joined['rfc'], left_index=True, right_index=True).reset_index(drop=False)
regr = pd.merge(joined['dtr'], joined['rfr'], left_index=True, right_index=True).reset_index(drop=False)

In [333]:
binclas.columns = pd.MultiIndex.from_tuples(('Classification', *col) for col in binclas.columns)
regr.columns = pd.MultiIndex.from_tuples(('Regression', *col) for col in regr.columns)

In [334]:
result = pd.concat([binclas, regr], axis=1)

In [335]:
def formatting(row):
    res = pd.Series()
    for label, value in row.items():
        if label.endswith('diff'):
            string = f'{value:.0e}' if abs(value) > 1e-9 else '0'
            if string[0] != '-':
                string = f' {string}'
            res[label] = string
        elif label.startswith('rate'):
            res[label] = f'{value:.2f}'[1:] if value > 0 else '0'
        elif label == 'p':
            res[label] = '$\\neq$' if value < 0.05 else ''
    return res

In [336]:
result[('Classification', 'Decision Tree')]

/tmp/ipykernel_2300987/4189619260.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Decision Tree')]


,auc_diff,p,rate,rate_kfold
0,0.000000,5.000000e-01,0.000000,0.003534
1,-0.000335,4.971797e-04,0.000000,0.020378
2,0.000004,4.119086e-01,0.000000,0.014851
3,0.000000,5.000000e-01,0.000000,0.032316
4,0.000000,5.000000e-01,0.000000,0.000000
5,0.000105,7.051001e-03,0.117647,0.095642
6,-0.003139,7.374231e-02,0.000000,0.014337
7,0.000000,5.000000e-01,0.000000,0.056667
8,0.000000,5.000000e-01,0.000000,0.000000
9,0.000000,5.000000e-01,0.000000,0.000000


In [337]:
result[('Classification', 'Decision Tree')] = result[('Classification', 'Decision Tree')].apply(formatting, axis=1)
result[('Classification', 'Random Forest')] = result[('Classification', 'Random Forest')].apply(formatting, axis=1)
result[('Regression', 'Decision Tree')] = result[('Regression', 'Decision Tree')].apply(formatting, axis=1)
result[('Regression', 'Random Forest')] = result[('Regression', 'Random Forest')].apply(formatting, axis=1)

/tmp/ipykernel_2300987/513803122.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Decision Tree')] = result[('Classification', 'Decision Tree')].apply(formatting, axis=1)
/tmp/ipykernel_2300987/513803122.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Decision Tree')] = result[('Classification', 'Decision Tree')].apply(formatting, axis=1)
/tmp/ipykernel_2300987/513803122.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Random Forest')] = result[('Classification', 'Random Forest')].apply(formatting, axis=1)
/tmp/ipykernel_2300987/513803122.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  result[('Classification', 'Random Forest')] = result[('Classification', 'Random Forest')].apply(formatting, axis=1)
/tmp/ipykernel_2300987/513803122.py:3: PerformanceWarning: indexing past lexsort dep

In [338]:
result

Classification                                        \
                      name Decision Tree                           
                                auc_diff       p rate rate_kfold   
0             appendicitis             0            0        .00   
1                 haberman        -3e-04  $\neq$    0        .02   
2             new_thyroid1         4e-06            0        .01   
3                   glass0             0            0        .03   
4         shuttle-6_vs_2-3             0            0          0   
5                     bupa         1e-04  $\neq$  .12        .10   
6         cleveland-0_vs_4        -3e-03            0        .01   
7                   ecoli1             0            0        .06   
8             poker-9_vs_7             0            0          0   
9                   monk-2             0            0          0   
10               hepatitis        -2e-04            0        .04   
11    yeast-0-3-5-9_vs_7-8        -4e-05            0        .04   
12            mammographic         3e-05  $\neq$    0        .00   
13                 saheart        -8e-06  $\neq$    0        .03   
14    page-blocks-1-3_vs_4         1e-04  $\neq$  .25        .12   
15            lymphography        -4e-03            0          0   
16                    pima        -3e-04  $\neq$    0        .05   
17               wisconsin        -1e-04  $\neq$    0        .03   
18             abalone9_18             0            0          0   
19  winequality-red-3_vs_5             0            0          0   

                                                     Regression                \
   Random Forest                                           name Decision Tree   
        auc_diff       p rate rate_kfold                              r2_diff   
0         -4e-06          .02        .03               diabetes        -7e-05   
1          6e-06          .04        .05                 o-ring        -4e-03   
2          8e-05  $\neq$  .07        .09        stock-portfolio        -1e-04   
3         -2e-05          .05        .05                wsn-ale        -1e-03   
4              0          .14        .13           daily-demand        -3e-04   
5          2e-03  $\neq$  .25        .24             slump_test         5e-04   
6         -8e-04  $\neq$  .09        .11                  servo        -7e-06   
7         -7e-05  $\neq$  .12        .12    yacht_hydrodynamics             0   
8         -2e-03  $\neq$  .14        .16               autoMPG6        -3e-04   
9              0          .00        .00     excitation_current        -4e-08   
10        -3e-04  $\neq$  .03        .04  real_estate_valuation        -4e-04   
11        -2e-04  $\neq$  .09        .09                wankara        -5e-05   
12         6e-07  $\neq$  .02        .04                plastic         1e-03   
13         6e-05  $\neq$  .07        .08                  laser        -9e-04   
14        -4e-06          .06        .06  qsar-aquatic-toxicity             0   
15        -1e-03  $\neq$  .04        .04               baseball        -1e-03   
16         4e-06          .16        .19   maternal_health_risk         1e-03   
17         4e-05          .09        .12        cpu_performance         2e-03   
18        -1e-06          .08        .08                airfoil         9e-06   
19        -4e-04  $\neq$  .07        .06           medical_cost             0   

                                                                  
                           Random Forest                          
         p rate rate_kfold       r2_diff       p rate rate_kfold  
0             0        .02         7e-04  $\neq$    0        .00  
1   $\neq$    0        .03        -2e-02  $\neq$  .13        .10  
2           .05        .03         2e-05  $\neq$  .02        .03  
3             0        .12        -1e-03  $\neq$  .12        .11  
4   $\neq$    0        .01        -7e-04  $\neq$  .03        .03  
5   $\neq$  .10        .09        -4e-04  $\neq

In [339]:
result[('Classification', 'name', '')] = result[('Classification', 'name', '')].apply(lambda x: x.replace('_', '-'))
result[('Regression', 'name', '')] = result[('Regression', 'name', '')].apply(lambda x: x.replace('_', '-'))

In [340]:
result

Classification                                        \
                      name Decision Tree                           
                                auc_diff       p rate rate_kfold   
0             appendicitis             0            0        .00   
1                 haberman        -3e-04  $\neq$    0        .02   
2             new-thyroid1         4e-06            0        .01   
3                   glass0             0            0        .03   
4         shuttle-6-vs-2-3             0            0          0   
5                     bupa         1e-04  $\neq$  .12        .10   
6         cleveland-0-vs-4        -3e-03            0        .01   
7                   ecoli1             0            0        .06   
8             poker-9-vs-7             0            0          0   
9                   monk-2             0            0          0   
10               hepatitis        -2e-04            0        .04   
11    yeast-0-3-5-9-vs-7-8        -4e-05            0        .04   
12            mammographic         3e-05  $\neq$    0        .00   
13                 saheart        -8e-06  $\neq$    0        .03   
14    page-blocks-1-3-vs-4         1e-04  $\neq$  .25        .12   
15            lymphography        -4e-03            0          0   
16                    pima        -3e-04  $\neq$    0        .05   
17               wisconsin        -1e-04  $\neq$    0        .03   
18             abalone9-18             0            0          0   
19  winequality-red-3-vs-5             0            0          0   

                                                     Regression                \
   Random Forest                                           name Decision Tree   
        auc_diff       p rate rate_kfold                              r2_diff   
0         -4e-06          .02        .03               diabetes        -7e-05   
1          6e-06          .04        .05                 o-ring        -4e-03   
2          8e-05  $\neq$  .07        .09        stock-portfolio        -1e-04   
3         -2e-05          .05        .05                wsn-ale        -1e-03   
4              0          .14        .13           daily-demand        -3e-04   
5          2e-03  $\neq$  .25        .24             slump-test         5e-04   
6         -8e-04  $\neq$  .09        .11                  servo        -7e-06   
7         -7e-05  $\neq$  .12        .12    yacht-hydrodynamics             0   
8         -2e-03  $\neq$  .14        .16               autoMPG6        -3e-04   
9              0          .00        .00     excitation-current        -4e-08   
10        -3e-04  $\neq$  .03        .04  real-estate-valuation        -4e-04   
11        -2e-04  $\neq$  .09        .09                wankara        -5e-05   
12         6e-07  $\neq$  .02        .04                plastic         1e-03   
13         6e-05  $\neq$  .07        .08                  laser        -9e-04   
14        -4e-06          .06        .06  qsar-aquatic-toxicity             0   
15        -1e-03  $\neq$  .04        .04               baseball        -1e-03   
16         4e-06          .16        .19   maternal-health-risk         1e-03   
17         4e-05          .09        .12        cpu-performance         2e-03   
18        -1e-06          .08        .08                airfoil         9e-06   
19        -4e-04  $\neq$  .07        .06           medical-cost             0   

                                                                  
                           Random Forest                          
         p rate rate_kfold       r2_diff       p rate rate_kfold  
0             0        .02         7e-04  $\neq$    0        .00  
1   $\neq$    0        .03        -2e-02  $\neq$  .13        .10  
2           .05        .03         2e-05  $\neq$  .02        .03  
3             0        .12        -1e-03  $\neq$  .12        .11  
4   $\neq$    0        .01        -7e-04  $\neq$  .03        .03  
5   $\neq$  .10        .09        -4e-04  $\neq

In [341]:
def rename_columns(col):
    if col == 'rate':
        return '$\\rho$'
    elif col == 'rate_kfold':
        return '$\\rho_{k}$'
    elif col == 'auc_diff':
        return 'auc$_{d}$'
    elif col == 'r2_diff':
        return 'r$^2_{d}$'
    else:
        return col

In [342]:
result.columns = pd.MultiIndex.from_tuples([(col[0], col[1], rename_columns(col[2]))
                                            for col in result.columns])

In [343]:
result

Classification                                          \
                      name Decision Tree                             
                               auc$_{d}$       p $\rho$ $\rho_{k}$   
0             appendicitis             0              0        .00   
1                 haberman        -3e-04  $\neq$      0        .02   
2             new-thyroid1         4e-06              0        .01   
3                   glass0             0              0        .03   
4         shuttle-6-vs-2-3             0              0          0   
5                     bupa         1e-04  $\neq$    .12        .10   
6         cleveland-0-vs-4        -3e-03              0        .01   
7                   ecoli1             0              0        .06   
8             poker-9-vs-7             0              0          0   
9                   monk-2             0              0          0   
10               hepatitis        -2e-04              0        .04   
11    yeast-0-3-5-9-vs-7-8        -4e-05              0        .04   
12            mammographic         3e-05  $\neq$      0        .00   
13                 saheart        -8e-06  $\neq$      0        .03   
14    page-blocks-1-3-vs-4         1e-04  $\neq$    .25        .12   
15            lymphography        -4e-03              0          0   
16                    pima        -3e-04  $\neq$      0        .05   
17               wisconsin        -1e-04  $\neq$      0        .03   
18             abalone9-18             0              0          0   
19  winequality-red-3-vs-5             0              0          0   

                                                       Regression  \
   Random Forest                                             name   
       auc$_{d}$       p $\rho$ $\rho_{k}$                          
0         -4e-06            .02        .03               diabetes   
1          6e-06            .04        .05                 o-ring   
2          8e-05  $\neq$    .07        .09        stock-portfolio   
3         -2e-05            .05        .05                wsn-ale   
4              0            .14        .13           daily-demand   
5          2e-03  $\neq$    .25        .24             slump-test   
6         -8e-04  $\neq$    .09        .11                  servo   
7         -7e-05  $\neq$    .12        .12    yacht-hydrodynamics   
8         -2e-03  $\neq$    .14        .16               autoMPG6   
9              0            .00        .00     excitation-current   
10        -3e-04  $\neq$    .03        .04  real-estate-valuation   
11        -2e-04  $\neq$    .09        .09                wankara   
12         6e-07  $\neq$    .02        .04                plastic   
13         6e-05  $\neq$    .07        .08                  laser   
14        -4e-06            .06        .06  qsar-aquatic-toxicity   
15        -1e-03  $\neq$    .04        .04               baseball   
16         4e-06            .16        .19   maternal-health-risk   
17         4e-05            .09        .12        cpu-performance   
18        -1e-06            .08        .08                airfoil   
19        -4e-04  $\neq$    .07        .06           medical-cost   

                                                                         \
   Decision Tree                           Random Forest                  
       r$^2_{d}$       p $\rho$ $\rho_{k}$     r$^2_{d}$       p $\rho$   
0         -7e-05              0        .02         7e-04  $\neq$      0   
1         -4e-03  $\neq$      0        .03        -2e-02  $\neq$    .13   
2         -1e-04            .05        .03         2e-05  $\neq$    .02   
3         -1e-03              0        .12        -1e-03  $\neq$    .12   
4         -3e-04  $\neq$      0        .01        -7e-04  $\neq$    .03   
5          5e-04  $\neq$    .10        .09        -4e-04  $\neq$    .18   
6         -7e-06              0          0        -9e-06  $\neq$    .00   
7              0              0          0             0       

In [344]:
latex = result.to_latex(
    index=False,
    multicolumn_format='c'
)

In [345]:
tabular_string = latex[len('\\begin{tabular}{'): len('\\begin{tabular}{') + len(result.columns)]
print(tabular_string)
tabular_string_new = 'lrlrrrlrrlrrrlrrrl'
updated = '@{\hspace{4pt}}'.join(tabular_string_new)
latex = latex.replace(tabular_string, updated)

llllllllllllllllll


In [346]:
print(latex)

\begin{tabular}{l@{\hspace{4pt}}r@{\hspace{4pt}}l@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}l@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}l@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}l@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}r@{\hspace{4pt}}l}
\toprule
\multicolumn{9}{c}{Classification} & \multicolumn{9}{c}{Regression} \\
name & \multicolumn{4}{c}{Decision Tree} & \multicolumn{4}{c}{Random Forest} & name & \multicolumn{4}{c}{Decision Tree} & \multicolumn{4}{c}{Random Forest} \\
 & auc$_{d}$ & p & $\rho$ & $\rho_{k}$ & auc$_{d}$ & p & $\rho$ & $\rho_{k}$ &  & r$^2_{d}$ & p & $\rho$ & $\rho_{k}$ & r$^2_{d}$ & p & $\rho$ & $\rho_{k}$ \\
\midrule
appendicitis &  0 &  & 0 & .00 & -4e-06 &  & .02 & .03 & diabetes & -7e-05 &  & 0 & .02 &  7e-04 & $\neq$ & 0 & .00 \\
haberman & -3e-04 & $\neq$ & 0 & .02 &  6e-06 &  & .04 & .05 & o-ring & -4e-03 & $\neq$ & 0 & .03 & -2e-02 & $\neq$ & .13 & .10 \\
new-thyroid1 &  4e-06 &  & 0 & .01 &  8e-05 & $\neq$

In [347]:
with open('tab_presence.tex', 'wt') as file:
    file.write(latex)